In [1]:
%load_ext literary.module

# Notebook Modules

Just providing importable notebook modules does not address the full scope of package development. When developing a package, we often want to perform relative imports `from . import x`. This requires that a `__package__` global is set to the name of the current package. Additionally, literary provides tools to make it easier to write literate documents e.g. `patch` which need to be exposed to the end user.

In [2]:
import sys
import warnings
from pathlib import Path

from traitlets import Enum
from traitlets.config import Configurable

from ..config import find_literary_config, load_literary_config
from ..transpile.patch import patch

Let's start by creating a configurable object which exposes a strategy for finding the best candidate for a package. This is determined either by the first entry in `sys.path` that contains a given file, or the shortest path from the package directory to the file.

In [3]:
class NotebookExtension(Configurable):
    package_name_strategy = Enum(
        ["shortest", "first"],
        default_value="shortest",
        help="strategy for resolving package name",
    ).tag(config=True)

Let's implement a function to determine the name of the current package

In [4]:
@patch(NotebookExtension)
def determine_package_name(self, path: Path) -> str:
    """Determine the corresponding importable name for a package directory given by
    a particular file path. Return `None` if path is not contained within `sys.path`.

    :param path: path to package
    :return:
    """
    if not path.is_dir():
        raise ValueError("Expected directory, not file path")

    candidates = []
    for p in sys.path:
        # If the cwd is only on `sys.path` exactly, then it can't
        # be a submodule. However, let's first look for parent paths
        if str(path) == p:
            continue

        # Resolve path relative to `sys.path`
        try:
            relative_path = path.relative_to(p)
        except ValueError:
            continue

        candidates.append(relative_path)

    # Nothing was found on `sys.path`, so we abort
    if not candidates:
        return

    # Different strategies for name resolution
    if self.package_name_strategy == "shortest":
        # This might be ambiguous (there may be several matches)
        # so let's take the most-direct path
        best_candidate = min(candidates, key=lambda x: len(x.parts))
    else:
        best_candidate = candidates[0]

    return ".".join(best_candidate.parts)

This package depends upon `nbformat`. Let's apply this algorithm to an `nbformat file`

In [5]:
import nbformat.corpus

In [6]:
extension = NotebookExtension()
# Find the path of nbformat.corpus from __path__
true_package_path = Path(nbformat.corpus.__path__[0])
package_name = extension.determine_package_name(true_package_path)

In [7]:
assert package_name == nbformat.corpus.__name__

Now let's plumb this feature in to the IPython extension.

In [8]:
def load_ipython_extension(ipython):
    """Build the package-aware namespace for the IPython kernel. 

    If the kernel working directory is located under `sys.path`, 
    the appropriate `__package__` global will be set.

    Additionally, helper utilities like `patch` are installed.

    :param ipython: IPython shell instance
    """
    cwd = Path.cwd()

    # Load the current project's config
    config = load_literary_config(find_literary_config(cwd))

    # Identify which package this belongs to
    notebook_extension = NotebookExtension(config=config)
    package = notebook_extension.determine_package_name(cwd)

    if package is None:
        warnings.warn(
            f"Couldn't determine the package name for the current working directory {cwd}. "
            f"This might be because the current project has not been installed in editable mode."
        )
        # Allow local absolute imports
        sys.path.append(str(cwd))
    else:
        # Packages don't have absolute imports
        # This should already have been done by the import hook, but let's guarantee
        sys.path = [p for p in sys.path if Path(p).resolve() != cwd]

    # Set `__package__` for consumer notebook
    # I don't think we should be setting this if we aren't a package,
    # but it seems like IPython sets it to None by default
    ipython.user_ns.update(
        {
            "__package__": package,
            "patch": patch,
        }
    )